In [ ]:
from create_dataset import ShipsEarDataset
from torchvision import datasets, transforms
import torchaudio
from AutoEncoder import FCEDN38

In [1]:
ANNOTATIONS_FILE="./Audio_annotations.csv"
AUDIO_DIR="./Data"
SAMPLE_RATE=44100
NUM_SAMPLES=5*Sfrom create_dataset import ShipsEarDataset
from torchvision import datasets, transforms
import torchaudioAMPLE_RATE
BATCH_SIZE=20

In [ ]:
spectrogram = torchaudio.transforms.Spectrogram(n_fft =1024)

In [ ]:
sed= ShipsEarDataset(ANNOTATIONS_FILE, AUDIO_DIR,spectrogram,SAMPLE_RATE,NUM_SAMPLES)

In [ ]:
val_size=int(0.1*len(sed))
test_size=int(0.1*len(sed))

In [ ]:
train_size= len(sed) - val_size- test_size
train_ds, val_ds,test_ds = random_split(sed,[train_size,val_size,test_size])
len(train_ds), len(val_ds),len(test_ds)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_dl=DataLoader(train_ds, 
                    batch_size=BATCH_SIZE, 
                    shuffle=True,
                    num_workers=8, 
                    pin_memory=True
                   )
val_dl=DataLoader(val_ds,
                  batch_size=BATCH_SIZE,
                  num_workers=8,
                  pin_memory=True
                 )
test_dl=DataLoader(test_ds,
                  batch_size=BATCH_SIZE,
                  num_workers=8,
                  pin_memory=True
                 )

In [ ]:
def train_model(model, train_loader, num_epochs=20, learning_rate=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        for signal,targets in train_loader:
            inputs = signal
            #inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
model=FCEDN38()

In [ ]:
train_dataloader= DeviceDataLoader(train_dl,device)
val_dataloader= DeviceDataLoader(val_dl,device)
test_dataloader= DeviceDataLoader(test_dl,device)
#to_device(model,device)
model=model.to(device)

In [ ]:
train_model(model, train_dataloader)